# This file is where the image classification model was trained and exported to TensorFlow Lite

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
"""
Used to load the train images as numpy arrays
"""
greendata = []
for i in range(1,10):
    x = 'Images/green/0000000'+str(i)+".jpg"
    try:
        plt.imshow(cv2.imread(x));
        assert cv2.imread(x).shape == (100,100,3)
        greendata.append(cv2.imread(x))
    except:
        pass
for i in range(10,100):
    x = 'Images/green/000000'+str(i)+".jpg"
    try:
        plt.imshow(cv2.imread(x)); 
        assert cv2.imread(x).shape == (100,100,3)
        greendata.append(cv2.imread(x))
    except:
         pass
for i in range(100,200):
    x = 'Images/green/00000'+str(i)+".jpg"
    try:
        plt.imshow(cv2.imread(x)); 
        assert cv2.imread(x).shape == (100,100,3)
        greendata.append(cv2.imread(x))
    except:
         pass
reddata = []
for i in range(1,10):
    x = 'Images/red/0000000'+str(i)+".jpg"
    try:
        plt.imshow(cv2.imread(x));
        assert cv2.imread(x).shape == (100,100,3)
        reddata.append(cv2.imread(x))
    except:
        pass
for i in range(10,100):
    x = 'Images/red/000000'+str(i)+".jpg"
    try:
        plt.imshow(cv2.imread(x)); 
        assert cv2.imread(x).shape == (100,100,3)
        reddata.append(cv2.imread(x))
    except:
         pass
for i in range(100,200):
    x = 'Images/red/00000'+str(i)+".jpg"
    try:
        plt.imshow(cv2.imread(x)); 
        assert cv2.imread(x).shape == (100,100,3)
        reddata.append(cv2.imread(x))
    except:
         pass
reddata = np.array(reddata)
greendata = np.array(greendata)
print(reddata.shape)
print(greendata.shape)

In [ ]:
"""
Shuffles two arrays (the images and the indices) in the same way so that corresponding values in each array are still the same as they initially were
"""
def same_shuffle(arr1,arr2):
    size = arr2.size
    print(size)
    idxs = np.arange(size)
    np.random.shuffle(idxs)
    return arr1[idxs], arr2[idxs]

In [ ]:
"""
Splits the data into train and validation sets
"""
greenlist = list(greendata)
redlist = list(reddata)
datalist = greenlist+redlist
data = np.array(datalist)
y_green = list(np.zeros(149))
y_red = list(np.zeros(128)+1)
y = np.array(y_green + y_red)
data, y = same_shuffle(data, y)
data_test = data[200:]
data = data[:200]
y_test = y[200:]
y = y[:200]
print(data_test.shape)
print(data.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
"""
Initializes the image classification neural network model in TensorFlow/Keras
"""
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(2, activation='softmax'))
model.summary()
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
"""
Trains the neural network on the train data and validates it on the validation set. Also saves the model as a .h5 file after each of the 50 epochs.
"""
filepath = 'weights.{epoch:02d}-{val_accuracy:.2f}.h5'
j = [tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)]
history = model.fit(data, y, epochs=50,validation_data=(data_test,y_test), callbacks=j)

In [ ]:
"""
Used to plot the loss and accuracy values for the model over the 50 epochs.
"""
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.plot(history.history['accuracy'], label='train accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.plot(history.history['loss'], label='train loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.xlim([0,21])
plt.legend(loc='lower right')

In [ ]:
# Saves the model
model.save('nn.h5')

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tflite_model = converter.convert()

In [ ]:
"""
Converts the model into a Float TensorFlow Lite model
"""
with open ('model.tflite', mode='wb') as file:
    file.write(tflite_model)

In [ ]:
"""
Converts the model into a Quantized TensorFlow Lite model
"""
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
with open ('detect_model.tflite', mode='wb') as file:
    file.write(tflite_quant_model)